Traing for time series without the output as an input

In [ ]:
from keras.activations import silu, swish, linear, tanh, leaky_relu, sigmoid, relu, elu, selu, gelu, linear
from keras.layers import Input, LSTM, Dense, GaussianNoise, BatchNormalization, Activation, PReLU, SimpleRNN
from keras.initializers.initializers_v2 import RandomNormal, RandomUniform
from keras.preprocessing.sequence import TimeseriesGenerator
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as ex
import plotly.io as pio
from keras.regularizers import l1, l2, L1L2
from keras.optimizers import nadam_v2, rmsprop_v2, adamax_v2
from keras.losses import mean_absolute_error as mae, mean_squared_error as mse, mean_absolute_percentage_error as mape
from keras.models import Sequential
import onnx
import tf2onnx
from keras.models import load_model
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from sklearn.model_selection import train_test_split
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA, KernelPCA,IncrementalPCA,SparsePCA
from joblib import dump, load
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from datetime import datetime
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import glob
import os
from scipy.signal import savgol_filter
import numpy as np
import seaborn as sn
%load_ext tensorboard


In [ ]:
model_version = 'v3.9.0'


In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


In [ ]:
# Directories for expreiments .csvs
source_dir = 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/combines/smoothed/With_cooling_feature_for_Model_training_5s/'
testdata_dir = 'G:/Innovations@HELLER/DN/KI/Temperaturkompensation/2021_Spindelwachstumskompensation_KI_HSU_SC63/Messungen DC100 H5000 M57002/row_files/combines/smoothed/With_cooling_feature_for_Model_training_5s/Testing_data/'
preprocessor_dir = 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\csvs\\smoothed\\AI_Model\\Backup\\Z_Welle\\' + model_version + '\\'
model_dir = 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\csvs\\smoothed\\AI_Model\\Backup\\Z_Welle\\' + model_version + '\\'
log_dir = "C:/Temperaturkompensation/logs/" + \
    datetime.now().strftime("%Y%m%d-%H%M%S")
weights_path = str(model_dir+"/weights3/weights")
image_path = 'G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\csvs\\smoothed\\AI_Model\\Backup\\Z_Welle\\' + model_version + '\\'


In [ ]:
from keras.losses import LossFunctionWrapper, mean_absolute_error, mean_squared_error
import tensorflow as tf
import keras.backend as k


def max_loss(y_true: tf.Tensor, y_pred: tf.Tensor):
    error = tf.multiply(tf.add(tf.multiply(tf.reduce_max(
        tf.abs((y_true - y_pred))), 1), mean_absolute_error(y_true, y_pred)), 1)
    return error


def max_loss_v1(y_true: tf.Tensor, y_pred: tf.Tensor):
    error = tf.add(tf.multiply(tf.reduce_max(tf.abs((y_true - y_pred))), 1), tf.sqrt(mean_squared_error(y_true, y_pred)))
    return error

def max_loss_v2(y_true: tf.Tensor, y_pred: tf.Tensor):
    error = tf.add(tf.multiply(tf.pow(tf.reduce_max(tf.abs((y_true - y_pred))),2), 1), mean_squared_error(y_true, y_pred))
    return error

def max_loss_v3_mae_std(y_true: tf.Tensor, y_pred: tf.Tensor):
    mean_ae= mean_absolute_error(y_true, y_pred)
    vars= tf.pow(tf.subtract(tf.abs(y_true - y_pred),mean_ae),2)
    errors_std= tf.sqrt(tf.reduce_mean(vars))
    #error = tf.add(mean_squared_error(y_true, y_pred), errors_std)
    error = tf.add(mean_ae, errors_std)
    return error

def loss_rmse_std(y_true: tf.Tensor, y_pred: tf.Tensor):
    mean_ae= mean_absolute_error(y_true, y_pred)
    vars= tf.pow(tf.subtract(tf.abs(y_true - y_pred),mean_ae),2)
    errors_std= tf.reduce_mean(vars)
    error = tf.add(mean_squared_error(y_true, y_pred), errors_std)
    #error = tf.add(mean_ae, errors_std)
    return error

def calc_fidelity(y_true: tf.Tensor, y_pred: tf.Tensor):
    # session= k.get_session()
    # y_true_array= session.run(y_true)
    # y_pred_array= session.run(y_pred)
    # cc= np.corrcoef(y_true_array,y_pred_array)
    NRMSE= tf.divide(tf.sqrt(mean_squared_error(y_true, y_pred)),tf.subtract(tf.reduce_max(y_true),tf.reduce_min(y_true)))
    fidelity= tf.abs(tf.multiply(100.0, tf.subtract(1.0,NRMSE)))
    return fidelity
    
special_loss = LossFunctionWrapper(loss_rmse_std)
fidelity_wrapper= LossFunctionWrapper(calc_fidelity)


In [ ]:
def generateDataSource_deprecated(signal_input=None, input_columns: list = [], output_length: int = 1, signal_output=None, window=1, shift=1):
    Signal_Length = signal_input.shape[0]
    num_samples = int((Signal_Length - window + 1) / shift)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i] = signal_input[i * shift: i * shift + window]
        y[i] = signal_output[i * shift + window -
                             1: i * shift + window - 1 + output_length]
    return x, y


In [ ]:
# Implement special preprocessing steps
# 1 calculate new Feature
# 2 make rounding


from sklearn.preprocessing import FunctionTransformer


def special_preprocess(data: np.ndarray):
    delta_t_spindel_bett = data[:, 1] - data[:, 0]
    delta_t_spindel_bett = delta_t_spindel_bett.reshape((-1, 1))
    result = np.concatenate((data[:, 2:5], delta_t_spindel_bett), axis=1)
    return result


def rounding(signals: np.ndarray):
    return np.round(signals, 2)


def remove_t_bett(data: np.ndarray):
    result = data[:, 1:]
    result[:, 0] = result[:, 0] - data[:, 0]
    result[:, 1] = result[:, 1] - data[:, 0]
    return result


data = np.asarray([[24, 90, 30, 500, 0, 0, 0, 0],
                   [24, 90, 30, 500, 0, 0, 0, 0],
                   [24, 90, 30, 500, 0, 0, 0, 0],
                   [24, 90, 30, 500, 0, 0, 0, 0]])
data.shape
print(remove_t_bett(data))
print(data)


In [ ]:
# selected_Columns= ['diff', 'drhz','t_bett',]#,'t_kluelung']# , 'diff'
# [ 't_bett','t_spindel', 't_y_achse', 't_z_achse', 'drhz', 't_motor','t_kss_tank', 't_raum', 't_spindel_vor', 't_spindel_rueck','t_schwenkantrieb']

output_variable = 'welle_z_iterpolated'#'welle_z_upsampled'#'z_welle_ok'  #'z_tcp_ok'# 'welle_z'#''#'smoothed_z_welle_ok'#
scaler = StandardScaler(with_mean=True, with_std=True)
minmax = MinMaxScaler(feature_range=(-1, 1))
pca = PCA(n_components=6)
#pca= IncrementalPCA(n_components=6)
# Data Parameters
selected_Columns = ['t_bett', 't_motor', 't_spindle', 'M8','M121', 'M127', 'M7']#['t_bett', 't_motor', 't_spindel', 'M8','M121', 'M127', 'M7']
#,('smoothing_data', FunctionTransformer(smoothing_data))('t_bett_removal', FunctionTransformer(remove_t_bett)),
proposed_pipline = Pipeline(steps=[ ('t_bett_removal', FunctionTransformer(remove_t_bett)),('rounding', FunctionTransformer(rounding)), ('stdscaler', scaler)])#('stdscaler', scaler), ('pca', pca)])  # Pipeline(steps=[('rounding', FunctionTransformer(rounding))])#('pca', pca)
preprocessor_name = "preprocessor.p"
postprocessor= 'postprocessor.p'
smoothed_output= False
window= 30
shift= 1


In [ ]:
def build_Model_LSTM(X_train):
    activation_Function = selu
    dropout = 0.01
    l1_v = 0.0008
    L2_v = 0.0008
    size = 50
    n_hidden_layers =4
    diff= 10
    unroll = False
    kernal_init = 'he_normal'#RandomUniform()  # 
    #optimizer = nadam_v2.Nadam(clipnorm= 1)
    optimizer= adamax_v2.Adamax(clipnorm= 1)
    model = Sequential()
    #activation = PReLU()
    model.add(LSTM(size, input_shape=(window, len(selected_Columns)-1), return_sequences=True,kernel_initializer= kernal_init,unroll=unroll, dropout=dropout, kernel_regularizer=L1L2(l1=l1_v, l2=L2_v)))
    #model.add(BatchNormalization())
    model.add(Activation(activation=activation_Function))
    #model.add(PReLU())
    for i in range(1, n_hidden_layers+1):
        if i== n_hidden_layers:
            model.add(LSTM(size - i * diff, return_sequences=False,unroll=unroll,kernel_initializer= kernal_init, dropout=dropout, kernel_regularizer=L1L2(l1=l1_v, l2=L2_v)))
        else:
            model.add(LSTM(size - i * diff, return_sequences=True,unroll=unroll, dropout=dropout,kernel_initializer= kernal_init,  kernel_regularizer=L1L2(l1=l1_v, l2=L2_v)))
        #model.add(BatchNormalization())
        model.add(Activation(activation=activation_Function))
        #model.add(PReLU())
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer=optimizer, loss=special_loss, metrics=[max_loss])
    return model

In [ ]:
def smoothing_data(input: pd.DataFrame, window= 12):
    # data= input.copy()
    # data['t_bett']= data['t_bett'].rolling(window= window).mean()
    # data['t_motor']= data['t_motor'].rolling(window= window).mean()
    # data['t_spindle']= data['t_spindle'].rolling(window= window).mean()
    # #data[output_variable]= data[output_variable].rolling(window= window).mean()
    # data= data[ window+1:]###Remove nan
    return input#data

Loading Training Data

In [ ]:
# # Extract Data source from all signals and create Training dataset
# working_dir = source_dir
# os.chdir(working_dir)
# results: np.ndarray = None
# targets: np.ndarray = None
# pipeline = None
# reset = True
# training_data = None
# #selected_Columns= ['t_motor','t_spindel', 'drhz']
# for file in glob.glob('*.csv'):
#     if file == 'combined.csv':
#         continue
#     print('Current File: ', file)
#     df = pd.read_csv(file)
#     if df.isnull().values.any():
#         print ('with nan')
#     signals = df
#     prev_time=None
#     drop_list= []
#     print('Before Filtering, ', len(signals))
#     for index,row in signals.iterrows():
#         current_time= datetime.strptime(row['date'], "%Y-%m-%d %H:%M:%S.%f")
#         if prev_time is None:
#             prev_time= current_time
#         elif (current_time - prev_time).total_seconds() < 5:
#             drop_list.append(datetime.strftime(current_time, "%Y-%m-%d %H:%M:%S.%f"))
#             df= df[df['date'] != row['date']]
#         prev_time= current_time
#     signals= df
#     signals= smoothing_data(signals)
#     print('After Filtering, ', len(signals), ', deleted= ', len(drop_list))
#     signals = signals[selected_Columns]
#     if signals.isnull().values.any():
#         print ('with nan')
#         exit()
#     #signals = preprocess_data(signals, kuelung=['M8', 'M121', 'M127', 'M7'])
#     signals= signals.to_numpy()
#     if training_data is None:
#         training_data = signals
#     else:
#         training_data = np.concatenate((training_data, signals))
# #training_data = training_data.to_numpy()
# if pipeline is None:
#     try:
#         if not reset:
#             pipeline = load(str(preprocessor_dir + preprocessor_name))
#             print('preprocessor is loaded successfully')
#         else:
#             pipeline = proposed_pipline
#             pipeline = pipeline.fit(training_data)
#             dump(pipeline, str(preprocessor_dir + preprocessor_name))
#             print('preprocessor is saved successfully')
#             print('explained_variance_ratio_= ', pipeline.steps[3][1].explained_variance_ratio_)
#     except FileNotFoundError as err:
#         pipeline = proposed_pipline
#         pipeline = pipeline.fit(training_data)
#         dump(pipeline, str(preprocessor_dir + preprocessor_name))
#         initial_type = [('float_input', FloatTensorType(
#             [None,  training_data.shape[1]]))]
#         onx = convert_sklearn(pipeline, initial_types=initial_type)
#         with open(str(preprocessor_dir + "preprocessor.onnx"), "wb") as f:
#             f.write(onx.SerializeToString())
#         print('preprocessor is saved successfully')
# columns = selected_Columns.copy()
# columns.append(output_variable)
# for file in glob.glob('*.csv'):
#     if file == 'combined.csv':
#         continue
#     print('Current File: ', file)
#     df = pd.read_csv(file)
#     signals = df
#     prev_time=None
#     drop_list= []
#     print('Before Filtering, ', len(signals))
#     for index,row in signals.iterrows():
#         current_time= datetime.strptime(row['date'], "%Y-%m-%d %H:%M:%S.%f")
#         if prev_time is None:
#             prev_time= current_time
#         elif (current_time - prev_time).total_seconds() < 5:
#             drop_list.append(datetime.strftime(current_time, "%Y-%m-%d %H:%M:%S.%f"))
#             df= df[df['date'] != row['date']]
#         prev_time= current_time
#     signals= df
#     signals= smoothing_data(signals)
#     print('After Filtering, ', len(signals), ', deleted= ', len(drop_list))
#     #signals = preprocess_data(signals, kuelung=['M8', 'M121', 'M127', 'M7'])
#     signals = signals[selected_Columns]
#     signals = signals.to_numpy()
#     print(signals.shape)
#     signals = pipeline.transform(signals)
#     output = 10000* df[output_variable]#.round(3)#(1000 * df[str('smoothed_' + output_variable)]).round(3)#1000* df[output_variable].round(3)#
#     partitions, target = generateDataSource(
#         signal_input=signals, input_columns=selected_Columns, output_length=1, signal_output=output, window=window, shift=shift)
#     if results is None:
#         results = partitions
#         targets = target
#     else:
#         results = np.concatenate((results, partitions), axis=0)
#         targets = np.concatenate((targets, target), axis=0)
#     # ####Replace negative values with 0
#     # targets[targets < 0] = 0
#     print('input: ', results.shape, ' output:', targets.shape)

In [ ]:
# # Extract Data source from all signals and create Training dataset
# data_path = str(source_dir + 'Test_area/')
# os.chdir(data_path)
working_dir = source_dir
os.chdir(working_dir)
results: np.ndarray = None
targets: np.ndarray = None
pipeline = None
reset = True
all_data = None
#selected_Columns= ['t_motor','t_spindel', 'drhz']
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df = pd.read_csv(file)
    if df.isnull().values.any():
        print ('with nan')
    signals = df
    prev_time=None
    drop_list= []
    print('Before Filtering, ', len(signals))
    for index,row in signals.iterrows():
        current_time= datetime.strptime(row['date'], "%Y-%m-%d %H:%M:%S.%f")
        if prev_time is not None and (current_time  - prev_time).total_seconds() < 5:
            drop_list.append(datetime.strftime(current_time, "%Y-%m-%d %H:%M:%S.%f"))
            df= df[df['date'] != row['date']]
            continue
        else:
            prev_time= current_time
    signals= df
    signals= smoothing_data(signals)
    print('After Filtering, ', len(signals), ', deleted= ', len(drop_list))
    if signals.isnull().values.any():
        print ('with nan')
        exit()
    ###Set index to data frame
    signals['date']= pd.to_datetime(signals['date'],format="%Y-%m-%d %H:%M:%S.%f")
    signals.set_index('date',inplace= True)
    #signals= signals[selected_Columns]
    ####Shifting label once forward#######
    signals[output_variable]= signals[output_variable].shift(+1)
    #print(all_data.iloc[2])
    signals= signals[1:]
    if all_data is None:
        all_data = signals
    else:
        all_data = pd.concat([all_data, signals])
targets= 1000 * all_data[output_variable]
all_data= all_data[selected_Columns]
#print(all_data.columns)
all_data= np.float32(all_data.to_numpy())
targets=    np.float32(targets.to_numpy().reshape((-1,1)))

Generate a Preprocessor

In [ ]:
if pipeline is None:
    try:
        if not reset:
            pipeline = load(str(preprocessor_dir + preprocessor_name))
            print('preprocessor is loaded successfully')
        else:
            pipeline = proposed_pipline
            pipeline = pipeline.fit(all_data)
            dump(pipeline, str(preprocessor_dir + preprocessor_name))
            print('preprocessor is saved successfully')
            #print('explained_variance_ratio_= ', pipeline.steps[3][1].explained_variance_ratio_)
    except FileNotFoundError as err:
        pipeline = proposed_pipline
        pipeline = pipeline.fit(all_data)
        dump(pipeline, str(preprocessor_dir + preprocessor_name))
        # initial_type = [('float_input', FloatTensorType(
        #     [None,  all_data.shape[1]]))]
        # onx = convert_sklearn(pipeline, initial_types=initial_type)
        # with open(str(preprocessor_dir + "preprocessor.onnx"), "wb") as f:
        #     f.write(onx.SerializeToString())
        print('preprocessor is saved successfully')

Create postprocessor

In [ ]:
post_processor= StandardScaler(with_mean=True, with_std=True)#MinMaxScaler(feature_range=(-1, 1))#Pipeline(steps=[('minmax', MinMaxScaler((-1,1)))])
post_processor= post_processor.fit(targets)#(all_data[: -1])
dump(post_processor, str(preprocessor_dir + 'postprocessor'))

In [ ]:
all_data[0:3]

In [ ]:
targets[0:3]

Apply preprocessing on data

In [ ]:
preprocessed_data= pipeline.transform(all_data)

In [ ]:
targets_transformed= post_processor.transform(targets)#preprocessed_data[:,-1]#

In [ ]:
x_train,x_test, y_train, y_test=train_test_split(preprocessed_data, targets_transformed, test_size=0.2, shuffle= False)

In [ ]:
# x_train= preprocessed_data
# y_train= preprocessed_data[:,-1]

Generate Data for LSTM

In [ ]:
x_train_windowed= TimeseriesGenerator(x_train,targets= y_train, length=window, sampling_rate= 1, batch_size= 1)
xtrain= np.asarray([x_train_windowed[i][0] for i in range(0, len(x_train_windowed))]).reshape((-1, window, len(selected_Columns)-1))
ytrain= np.asarray([x_train_windowed[i][1] for i in range(0, len(x_train_windowed))])
x_test_windowed= TimeseriesGenerator(x_test,targets= y_test, length=window, sampling_rate= 1, batch_size= 1)
xtest= np.asarray([x_test_windowed[i][0] for i in range(0, len(x_test_windowed))]).reshape((-1, window, len(selected_Columns)-1))
ytest= np.asarray([x_test_windowed[i][1] for i in range(0, len(x_test_windowed))])

In [ ]:
xtrain.shape

In [ ]:
ytrain.shape

In [ ]:
for i in range (10):
    print('i= ',i)
    print (xtrain[i],' label= ',ytrain[i] )
    print('#########################################################')

In [ ]:
xtest.shape

In [ ]:
ytest.shape

In [ ]:
def calculate_weights(y_train: np.ndarray, occurance_threshold= 0):
    y_train_rounded = np.round(y_train)
    N = len(y_train_rounded)
    y_train_rounded = np.reshape(y_train_rounded, newshape=(N,))
    weights = np.ones(shape=(N,))
    for i in range(0, N):
        current_value = y_train_rounded[i]
        weights[i]= (1 if current_value <=25 else 3.0 * current_value)
    fig= make_subplots(rows=2,cols=1,shared_xaxes= True, print_grid= True,  vertical_spacing=0.02)
    fig.add_trace(go.Line(y=weights,name='weight of Labels')#visualisation_selected_Columns[-2])
    , row= 1, col= 1)
    fig.add_trace(go.Line(y=np.reshape(y_train, newshape=(N,)),name='Labels')#visualisation_selected_Columns[-2])
    , row= 2, col= 1)
    # fig.update_yaxes(title_text= 'weight', row= 1, col= 1)
    # fig.update_xaxes(title_text= 'label value', row= 1, col= 1)
    fig.update_layout(height=900, width=900, title_text= 'weight of Labels')
    fig.show()

    return weights


In [ ]:
# Training Block
checkpoint = ModelCheckpoint(filepath=str(model_dir+'best_model.h5'), monitor='val_loss', verbose=1,
                             save_best_only=True, save_weights_only=False, mode='min', save_freq='epoch')
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)
earlyStopping = EarlyStopping(
    monitor='val_loss', mode='min', patience=200, verbose=1)
model = build_Model_LSTM(X_train=xtrain)
#reset= False
try:
    if not reset:
        model.load_weights(filepath=weights_path)
        print('Previous weights loaded Successfully')
except:
    print('No Previous weights')
print('input shape ', model.input_shape)
print(model.output_shape)
print(model.summary())
summary = model.fit(x= xtrain,y= ytrain, shuffle=False,  epochs=5000,batch_size=100 , validation_data= (xtest, ytest),  #validation_split=0.2, #
                    callbacks=[earlyStopping, checkpoint], verbose=2, workers=32, use_multiprocessing=True)#, sample_weight=weights)
model.save(str(model_dir+'model.h5'), save_format='h5')
input_signature = [tf.TensorSpec(
    [None, xtrain.shape[1], xtrain.shape[2]], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(
    model=model, input_signature=input_signature, opset=10)
onnx.save_model(onnx_model, str(model_dir+'model.onnx'))
print(".onnx model saved successfully")

In [ ]:
# model.save_weights(weights_path, save_format='h5')
# model.save_weights(weights_path, save_format='tf')
# model.save(str(model_dir+'testmodel.h5'), save_format='h5', overwrite=True)

In [ ]:
#model.save_weights(filepath=weights_path)
loss = summary.history['loss']
val_loss = summary.history['val_loss']  
fig = make_subplots(rows=1, cols=1, shared_xaxes=True,
                    print_grid=True,  vertical_spacing=0.02)
fig.add_trace(go.Line(y=loss, name='Training Loss'), row=1, col=1)
fig.add_trace(go.Line(y=val_loss, name='Validation Loss'), row=1, col=1)
fig.update_xaxes(title_text='Epochs', row=1, col=1)
fig.update_yaxes(title_text='Max_Loss', row=1, col=1)
fig.update_layout(height=900, width=900, title_text='training_curve')
fig.show()
pio.write_image(fig, str(image_path+'training_curve.png'), format='png')

In [ ]:
# #####Save Model as onnx from weights
# from keras.models import load_model
# import tf2onnx
# import onnx
# import tensorflow as tf
# model= Sequential()
# model= load_model(filepath= str(model_dir+'best_model.h5'), compile= False)
# #model.load_weights(weights_path)
# #model.save(str(model_dir+'best_model.h5'), save_format='h5')
# input_signature= [tf.TensorSpec([None,240, 6],tf.float32, name= 'x')]
# onnx_model, _= tf2onnx.convert.from_keras(model= model, input_signature= input_signature, opset= 10)
# onnx.save_model(onnx_model, str(model_dir+'Best_model.onnx'))
# print (".onnx model saved successfully")


In [ ]:
def post_process(input: np.ndarray, output: np.ndarray, scaler: MinMaxScaler):
    temp_array= np.zeros_like(input)
    output_shape= input[:,-1].shape
    temp_array[:,-1]= np.reshape(output, newshape= output_shape)
    reversed_input= scaler.inverse_transform(temp_array)
    post_processed_output=  reversed_input[:, -1]
    return post_processed_output

In [ ]:
import numpy as np
tolerance = 5
x_steps = np.linspace(0, 50, 10)
lowerline = x_steps - tolerance * np.sin(np.pi/4)
upperline = x_steps + tolerance * np.sin(np.pi/4)
# Create Tolerance Lines
tolerance = 10
x_steps = np.linspace(0, 50, 10)
lowerline_actual = x_steps - tolerance * np.sin(np.pi/4)
upperline_actual = x_steps + tolerance * np.sin(np.pi/4)


In [ ]:
import numpy as np
from keras.models import load_model
#from keras.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
# model= build_Model(X_train= results,y_train= targets)
# model.load_weights(filepath= weights_path)
# model.save(str(model_dir+'model.h5'), save_format='h5')
model = Sequential()
model = load_model(filepath=str(model_dir+'model.h5'), compile=False)
pipeline = load(str(preprocessor_dir+preprocessor_name))
print(model.summary())
print(model.input_shape)
# pred_train= model.predict(x_train)
# print ('mse= ',np.mean(mean_squared_error(y_train,pred_train)), ' mae= ', np.mean(mean_absolute_error(y_train,pred_train)), ' mape= ', np.mean(mean_absolute_percentage_error(y_train,pred_train)))
# pred_test= model.predict(x_test)
# print ('mse= ',np.mean(mean_squared_error(y_test,pred_test)), ' mae= ', np.mean(mean_absolute_error(y_test,pred_test)), ' mape= ', np.mean(mean_absolute_percentage_error(y_test,pred_test)))
pred = model.predict(xtrain)
pred_postprocessed =  post_processor.inverse_transform(pred)
true_postprocessed = post_processor.inverse_transform(y_train[window :].reshape(pred.shape))
# pred_postprocessed = post_process(for_post_process_input, pred, minmax)
# true_postprocessed = post_process(for_post_process_input, y_train[window :].reshape(pred.shape), minmax)
#print (' For All Data mse= ',mean_squared_error(targets,pred), ' mae= ', mean_absolute_error(targets,pred), ' mape= ', mean_absolute_percentage_error(targets,pred))
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Results of LSTM Algorithm')
plt.xlabel('True')
plt.ylabel('Predicted')

plt.plot(x_steps, upperline, label='Upper Bound', color='green')
plt.scatter(true_postprocessed, pred_postprocessed, label='predictions_train', color='blue')
plt.plot(x_steps, lowerline, label='lower Bound', color='green')
plt.plot(x_steps, x_steps, label='Optimal line')
plt.plot(x_steps, lowerline_actual, label='Actual lower Bound', color='red')
plt.plot(x_steps, upperline_actual, label='Actual Upper Bound', color='red')
plt.legend()
plt.savefig(str(image_path+'training_results.png'))
plt.show()


In [ ]:
# #####Save Model as onnx from weights
# from keras.models import load_model
# import tf2onnx
# import onnx
# import tensorflow as tf
# model= Sequential()
# model= load_model(filepath= str(model_dir+'model.h5'), compile= False)
# model.load_weights(weights_path)
# model.save(str(model_dir+'best_model.h5'), save_format='h5')
# input_signature= [tf.TensorSpec([None,results.shape[1], results.shape[2]],tf.float32, name= 'x')]
# onnx_model, _= tf2onnx.convert.from_keras(model= model, input_signature= input_signature, opset= 10)
# onnx.save_model(onnx_model, str(model_dir+'Best_model.onnx'))
# print (".onnx model saved successfully")


In [ ]:
import numpy as np
from keras.models import load_model
#from keras.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error, mean_squared_error
model = Sequential()
model = load_model(filepath=str(model_dir+'model.h5'), compile=False)
pipeline = load(str(preprocessor_dir+preprocessor_name))
# print(model.summary())
# pred_train= model.predict(x_train)
# print ('mse= ',np.mean(mean_squared_error(y_train,pred_train)), ' mae= ', np.mean(mean_absolute_error(y_train,pred_train)), ' mape= ', np.mean(mean_absolute_percentage_error(y_train,pred_train)))
# pred_test= model.predict(x_test)
# print ('mse= ',np.mean(mean_squared_error(y_test,pred_test)), ' mae= ', np.mean(mean_absolute_error(y_test,pred_test)), ' mape= ', np.mean(mean_absolute_percentage_error(y_test,pred_test)))
pred = model.predict(xtest)
pred_postprocessed =  post_processor.inverse_transform(pred)
true_postprocessed = post_processor.inverse_transform(y_test[window :].reshape(pred.shape))
# for_post_process_input= xtest[:, window-1,:]#.reshape((-1,len(selected_Columns)-1))
# pred_postprocessed = post_process(for_post_process_input, pred, minmax)
# true_postprocessed = post_process(for_post_process_input, y_test[window :].reshape(pred.shape), minmax)
#print (' For All Data mse= ',mean_squared_error(targets,pred), ' mae= ', mean_absolute_error(targets,pred), ' mape= ', mean_absolute_percentage_error(targets,pred))
fig, ax = plt.subplots(figsize=(15, 15))
plt.title('Ergebnisse des KI-Modells')
plt.xlabel('Gemessene Verlagerung (Welle) Mm')
plt.ylabel('Vorhersage der Verlagerung (Welle) Mm')
plt.plot(x_steps, upperline, label='5 Mm Obere Grenze', color='green')
plt.scatter(true_postprocessed, pred_postprocessed, label='Vorhersage', color='blue')
# plt.scatter(y_train,pred_train, label= 'predictions_train', color= 'blue')
# plt.scatter(y_test,pred_test, label= 'predictions_test', color= 'magenta')
plt.plot(x_steps, lowerline, label='5 Mm Untere Grenze', color='green')
plt.plot(x_steps, x_steps, label='Optimale Linie')
plt.plot(x_steps, lowerline_actual, label='10 Mm Obere Grenze', color='red')
plt.plot(x_steps, upperline_actual, label='10 Mm Untere Grenze', color='red')
plt.legend()
plt.savefig(str(image_path+'testing_results.png'))
plt.show()


Apply Predictions on experiments sqentially

In [ ]:
time_relevance=5
recording_time= 5
response_time= 5

In [ ]:
def extract_time_relevance_array(input_array: np.ndarray, output_shape):
    input_time= input_array.shape[1] # in case of 5 Seconds it will be 240 seconds
    output_time= output_shape[1] # in case of 2 Minutes it will be 10
    step= int(input_time/output_time)
    resut_array= np.zeros(output_shape)
    for i in range( 0, output_time):
        #print('place in result: ', output_time - i - 1, ' place in original: ', input_time - i*step -1)
        resut_array[0, output_time - i - 1, :]= input_array[0, input_time - i*step -1,:]
    return resut_array

In [ ]:
##### Simulate the prediction by passing values with 5 seconds ########
inputsignal_list= ['t_bett', 't_motor', 't_spindle', 'M8','M121', 'M127', 'M7']
custom_date_parser = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f")
num_features= len(inputsignal_list)
data_path = str(source_dir + 'Test_area/')
os.chdir(data_path)
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df = pd.read_csv(file, parse_dates= True, date_parser= custom_date_parser)
    if df.isnull().values.any():
        print ('with nan')
    input_data= df.copy()
    preds= []
    shape=(1,window,num_features +1)
    output_sec_shape= (1, int(window*np.ceil((time_relevance/recording_time))), num_features + 1)
    first_run= True
    last_prediction_time= None
    outputs= 0
    prev_prediction=0
    prev_record_time= None
    j= -10
    for index, data_point in input_data.iterrows():
        current_time= datetime.strptime(data_point['date'] , "%Y-%m-%d %H:%M:%S.%f")
        if prev_record_time is not None and (current_time  - prev_record_time).total_seconds() < recording_time:
            #print('Escaped! current_time: ',current_time, ' prev_record_time: ',prev_record_time )
            #preds.append(out)
            df= df[df['date']!= data_point['date']]
            continue
        else:
            prev_record_time= current_time
        #print ('index: ', index, ', Date: ', data_point['date'])
        new_signal_list = [float(data_point[sig]) for sig in inputsignal_list]
        new_signal_list.append(prev_prediction)
        #print ('New_signal_list: ', new_signal_list, ' timestamp: ', data_point['date'])
        signals_ = pipeline.transform(np.asarray([new_signal_list]))#np.asarray([new_signal_list])#
        print ('signals_: ', signals_, ' timestamp: ', data_point['date'])
        if first_run:
            # Auffüllen der Struktur - Bei erstem Start ist nur ein Datensatz vorhanden,
            # dieser wird mit np.full in alle kopiert
            five_sec_array= np.float32(np.full(output_sec_shape, signals_[0]))
            array= five_sec_array#extract_time_relevance_array(five_sec_array,shape)
            outputs =model.predict(array)
            prev_prediction= outputs[0][0]#1000 *data_point[output_variable]#
            out = post_processor.inverse_transform(outputs)[0][0] / 1000
            print('prev_prediction : ', prev_prediction, 'out: ', out)
            preds.append(out)
            first_run = False
            last_prediction_time= current_time
            #print('prediction_time: ', data_point['date'])
            #continue
        elif (current_time - last_prediction_time).total_seconds()< response_time:
                print('No prediction. Just shift')
                five_sec_array [0][:-1] = five_sec_array[0][1:]
                five_sec_array[0, -1, :] = signals_[0]
                preds.append(out)
                #continue 
        else:
            #print("....Shifting the 5 seconds buffer and insert the new 5s-value")
            five_sec_array [0][:-1] = five_sec_array[0][1:]
            five_sec_array[0, -1, :] = signals_[0]
            array= five_sec_array#extract_time_relevance_array(five_sec_array, shape)
            #array = np.float32(array)
            ## Apply predicitions on 2 minutes buffer
            outputs =model.predict(array)
            prev_prediction= outputs[0][0]#1000 * data_point[output_variable]#outputs[0][0]
            out = post_processor.inverse_transform(outputs)[0][0] / 1000
            print('prev_prediction : ', prev_prediction, 'out: ', out)
            preds.append(out)
            last_prediction_time= current_time
            #print('prediction_time: ', data_point['date'])
        j+= 1
        print ('counter= ', j )
        print('buffer: ', array, ' Predction= ', prev_prediction)
        print('#########################################################################')
    df['local_preds']= preds # mm
    scatter_mode= 'lines'#'lines'# 'lines+markers'# 'markers'
    selected_Columns= ['t_bett','t_motor', 't_spindle' , 'M8', 'M121', 'M127', 'M7']
    local_prediction_error= df[output_variable] - df['local_preds']
    y_axis_names= selected_Columns
    signals= df
    #print(preds)
    fig= make_subplots(rows=len(selected_Columns)+2 ,cols=1,shared_xaxes= True, print_grid= True, subplot_titles= selected_Columns, vertical_spacing=0.02)
    for i in range(len(selected_Columns)):
        fig.add_trace(go.Scatter(x= signals['date'], y= signals[selected_Columns[i]], name=selected_Columns[i], mode= scatter_mode), row= i+1, col= 1)
        fig.update_yaxes(title_text= y_axis_names[i], row= i+1, col= 1)
    ##Draw the prediciton and the real values of displacement on Welle
    fig.add_trace(go.Scatter(x= signals['date'],y= 1000 * signals[output_variable], name=output_variable, mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
    fig.add_trace(go.Scatter(x= signals['date'], y=  1000 * signals['local_preds'], name='Local predictions', mode= scatter_mode),  row= len(selected_Columns)+1, col= 1)
    fig.update_yaxes(title_text= output_variable, row= len(selected_Columns), col= 1)
    fig.add_trace(go.Scatter(x= signals['date'], y=   1000 * local_prediction_error, name='New Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+2 , col= 1)
    ## Draw the tolerence +-5
    fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(local_prediction_error,5), name='+5 Max Error', mode= scatter_mode),  row= len(selected_Columns)+2 , col= 1)
    fig.add_trace(go.Scatter(x= signals['date'], y= np.full_like(local_prediction_error,-5), name='-5 Min Error', mode= scatter_mode),  row= len(selected_Columns)+2 , col= 1)
    #fig.add_trace(go.Scatter(x= signals['date'], y=   1000*( np.abs(signals[output_variable] - signals[prediction_variable])), name='Old Model prediction Error', mode= scatter_mode),  row= len(selected_Columns)+1 , col= 1)
    fig.update_yaxes(title_text= 'Error (Micro-meter)', row= len(selected_Columns)+2, col= 1)
    fig.update_layout(height=1200, width=1400, title_text= 'Prediction Results')
    #pio.write_image(fig, str(images_path +'versuch_25_09_iso.png'), format='png')
    fig.show()


In [ ]:
#image_path= str(source_dir + 'results/')
#selected_Columns= ['t_spindel', 't_motor']
# Validate the model based on validataion data
data_path = str(source_dir + 'validate_Data/')
#image_path= str(source_dir + 'results/')
pipeline = load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
model = Sequential()
model = load_model(str(model_dir+'model.h5'), compile=False)
print('Model is loaded successfully')
os.chdir(data_path)
errors = []
error_df = pd.DataFrame()
scatter_mode= 'lines'
visualisation_selected_Columns= ['t_bett', 't_motor', 't_spindel', 'M8','M121', 'M127', 'M7',output_variable, 'prediction_Error' ]
for file in glob.glob('*.csv'):
    if file == 'combined.csv':
        continue
    print('Current File: ', file)
    df = pd.read_csv(file)
    if df.isnull().values.any():
        print ('with nan')
    signals = df
    prev_time=None
    drop_list= []
    print('Before Filtering, ', len(signals))
    for index,row in signals.iterrows():
        current_time= datetime.strptime(row['date'], "%Y-%m-%d %H:%M:%S.%f")
        if prev_time is None:
            prev_time= current_time
        elif (current_time - prev_time).total_seconds() < 5:
            drop_list.append(datetime.strftime(current_time, "%Y-%m-%d %H:%M:%S.%f"))
            df= df[df['date'] != row['date']]
        prev_time= current_time
    signals= df
    signals= smoothing_data(signals)
    print('After Filtering, ', len(signals), ', deleted= ', len(drop_list))
    if signals.isnull().values.any():
        print ('with nan')
        exit()
    ###Set index to data frame
    signals['date']= pd.to_datetime(signals['date'],format="%Y-%m-%d %H:%M:%S.%f")
    signals.set_index('date',inplace= True)
    signals= signals[selected_Columns]
    
    # (1000 * df[str('smoothed_' + output_variable)]).round(2)#
    output = (1000* df[output_variable]).round(3)#(1000 * df[str('smoothed_' + output_variable)]).round(3)
    partitions, target = generateDataSource(
        signal_input=signals, input_columns=selected_Columns, output_length=1, signal_output=output, window=window, shift=1)
    target = target.reshape((-1,))
    predictions = model.predict(partitions)
    predictions = np.reshape(predictions, newshape=target.shape)
    diff = np.abs(target - predictions)
    mae_v = ' mae= ' + str(np.mean(mean_absolute_error(target, predictions).round(2))) +' Micro_Meter '  
    max_v = ' max= ' + str(np.max(np.abs(target - predictions).round(2))) +' Micro_Meter '  
    fidelity= ' Fedility= ' +str(calc_fidelity_metric(target, predictions).round(2))+"%"
    print(mae_v)
    print(max_v)
    print(fidelity)
    ######################
    # Visualization of lf-data with predicted results
    visualisation_selected_Columns = selected_Columns.copy()
    visualisation_selected_Columns.append(output_variable)
    visualisation_df = df[window-1:]
    visualisation_df['predictions'] = predictions
    fig = make_subplots(rows= 1, cols=1, shared_xaxes=True, print_grid=True,  vertical_spacing=0.02)
    ###verlagerung
    fig.add_trace(go.Line(y= 1000 * visualisation_df[output_variable], name='Echte z_welle'), row= 1, col=1)
    fig.add_trace(go.Line(y= visualisation_df['predictions'], name='KI_z_welle'), row= 1, col=1)
    fig.update_yaxes(title_text='Verlagerung Mic-Meter', row= 1, col=1)
    fig.update_xaxes(title_text='Zeit (1= 2 Minuten)', row=1, col=1)
    fig.update_layout(height=900, width=1200, title_text=str(file + mae_v + max_v+ fidelity))
    fig.show()
    #pio.write_image(fig, str(image_path + file+'_Verlagerung.png'), format='png')


In [ ]:
#image_path= str(source_dir + 'results/')
#selected_Columns= ['t_spindel', 't_motor']
# Validate the model based on validataion data
data_path = str(source_dir + 'validate_Data/')
#image_path= str(source_dir + 'results/')
pipeline = load(str(preprocessor_dir+preprocessor_name))
print('preprocessor is loaded successfully')
#model = build_Model_LSTM()
model = load_model(str(model_dir+'best_model.h5'), compile=False)
print('Model is loaded successfully')
os.chdir(data_path)
errors = []
error_df = pd.DataFrame()
scatter_mode= 'lines'
visualisation_selected_Columns= ['t_bett', 't_motor', 't_spindle', 'M8','M121', 'M127', 'M7',output_variable, 'prediction_Error' ]
for file in glob.glob('*.csv'):
    df = pd.read_csv(file)
    #df = remove_peeks(df, output_variable, 4)
    print('current File: ', file)
    #df['t_spindel']= df['t_spindle']
    #signals = preprocess_data(df, kuelung=['M8', 'M121', 'M127', 'M7'])
    signals= df
    prev_time=None
    drop_list= []
    print('Before Filtering, ', len(signals))
    for index,row in signals.iterrows():
        current_time= datetime.strptime(row['date'], "%Y-%m-%d %H:%M:%S.%f")
        if prev_time is None:
            prev_time= current_time
        elif (current_time.second - prev_time.second) == 0:
            drop_list.append(datetime.strftime(current_time, "%Y-%m-%d %H:%M:%S.%f"))
            df= df[df['date'] != row['date']]
        prev_time= current_time
    signals= df
    print('After Filtering, ', len(signals), ', deleted= ', len(drop_list))
    signals = df[selected_Columns]
    signals = signals.to_numpy()
    signals = pipeline.transform(signals)
    # (1000 * df[str('smoothed_' + output_variable)]).round(2)#
    output = (1000* df[output_variable]).round(3)#(1000 * df[str('smoothed_' + output_variable)]).round(3)
    partitions, target = generateDataSource(
        signal_input=signals, input_columns=selected_Columns, output_length=1, signal_output=output, window=window, shift=1)
    target = target.reshape((-1,))
    predictions = model.predict(partitions)
    predictions = np.reshape(predictions, newshape=target.shape)
    diff = np.abs(target - predictions)
    mae_v = ' mae= ' + str(np.mean(mean_absolute_error(target, predictions)).round(2)) +' Micro_Meter '  
    max_v = ' max= ' + str(np.max(np.abs(target - predictions)).round(2)) +' Micro_Meter '  
    #fidelity= ' Fedility= ' +str(calc_fidelity_metric(target, predictions).round(2))+"%"
    ######################
    # ####Apply enterpolation###
    # upsampled_variable= output_variable#str('interpolated_'+output_variable)
    # signals[upsampled_variable]= signals[output_variable]
    # current_welle_z= 1000000
    # for index, row in signals.iterrows():
    #     if row[upsampled_variable]!= current_welle_z: ### A New value is available
    #         current_welle_z= row[upsampled_variable]
    #     else:
    #         signals.at[index,upsampled_variable]= np.NaN
    # signals[upsampled_variable].interpolate(method='linear', order=3, axis= 0,inplace= True)
    # output=  signals[output_variable]
    #######################
    # Visualization of lf-data with predicted results
    visualisation_selected_Columns = selected_Columns.copy()
    visualisation_selected_Columns.append(output_variable)
    visualisation_df = df[window-1:]
    visualisation_df[str('smoothed_' + output_variable)] = output
    visualisation_df['predictions'] = predictions
    fig = make_subplots(rows= 9, cols=1, shared_xaxes=True, print_grid=True,  vertical_spacing=0.02)
    # ##t_bett
    # fig.add_trace(go.Line(y=visualisation_df['t_bett'], name='t_bett'), row= 1, col=1)
    # fig.update_yaxes(title_text='t_bett C°', row= 1, col=1)
    # ##t_bett
    # fig.add_trace(go.Line(y=visualisation_df['drhz'], name='drhz'), row= 1, col=1)
    # fig.update_yaxes(title_text='drhz RPM', row= 1, col=1)
    ###t_motor
    fig.add_trace(go.Line(y=visualisation_df['t_motor'], name='t_motor'), row= 2, col=1)
    fig.update_yaxes(title_text='t_motor C°', row= 2, col=1)
    ###t_spindel
    fig.add_trace(go.Line(y=visualisation_df['t_spindle'], name='t_spindle'), row= 3, col=1)
    fig.update_yaxes(title_text='t_spindel C°', row= 3, col=1)
    ###M8
    fig.add_trace(go.Line(y=visualisation_df['M8'], name='M8'), row= 4, col=1)
    fig.update_yaxes(title_text='M8', row= 4, col=1)
    ###M121
    fig.add_trace(go.Line(y=visualisation_df['M121'], name='M121'), row= 5, col=1)
    fig.update_yaxes(title_text='M121', row= 5, col=1)
    ###M127
    fig.add_trace(go.Line(y=visualisation_df['M127'], name='M127'), row= 6, col=1)
    fig.update_yaxes(title_text='M127', row= 6, col=1)
    ###M7
    fig.add_trace(go.Line(y=visualisation_df['M7'], name='M7'), row= 7, col=1)
    fig.update_yaxes(title_text='M7', row= 7, col=1)
    ###verlagerung
    fig.add_trace(go.Line(y= 1000 * visualisation_df[output_variable], name='z_welle_Soll'), row= 8, col=1)
    fig.add_trace(go.Line(y= visualisation_df['predictions'], name='KI_z_welle_Ist'), row= 8, col=1)
    fig.update_yaxes(title_text='Verlagerung Mic-Meter', row= 8, col=1)
    ###Fehler
    experiment_errors = (1000*visualisation_df[output_variable] - visualisation_df['predictions']).to_list()
    fig.add_trace(go.Scatter( y=   experiment_errors, name='Prediction Error', mode= scatter_mode),  row= 9 , col= 1)
    ## Draw the tolerence +-5
    fig.add_trace(go.Scatter(y= np.full_like(experiment_errors,5), name='+5 Max Error', mode= scatter_mode),  row= 9 , col= 1)
    fig.add_trace(go.Scatter(y= np.full_like(experiment_errors,-5), name='-5 Min Error', mode= scatter_mode),  row= 9 , col= 1)
    fig.update_yaxes(title_text='Fehler Mic-Meter', row= 9, col=1)
    fig.update_xaxes(title_text='Zeit (1= 2 Minuten)', row=9, col=1)
    fig.update_layout(height=900, width=1200, title_text=str(file + mae_v + max_v))
    #fig.show()
    pio.write_image(fig, str(image_path + file+'.png'), format='png')
    current_error_df = pd.DataFrame(
        {'verlagerung': visualisation_df['predictions'].to_list(), 'error': experiment_errors})
    if error_df is None:
        error_df = current_error_df
    else:
        error_df = pd.concat([error_df, current_error_df])
    errors.extend(experiment_errors)


In [ ]:
# Visualization of resulted Data set
import plotly.express as px
labels = error_df['verlagerung']
verlagerung_df = pd.DataFrame({'verlagerung': labels})
print(verlagerung_df.head(5))
fig = px.histogram(verlagerung_df, x='verlagerung', nbins=100)
mean = verlagerung_df['verlagerung'].mean()
std = verlagerung_df['verlagerung'].std()
fig.update_layout(height=900, width=900, title_text=str(
    'Mean= ' + str(mean) + ', std=' + str(std)))
fig.show()
pio.write_image(
    fig, str(image_path + 'distribution of displacement.png'), format='png')


In [ ]:
# Visualization of resulted Data set
import plotly.express as px
error_df['abs_error'] = error_df['error'].abs()
print(error_df.head(5))
fig = px.histogram(error_df, x='verlagerung',
                   y='abs_error', nbins=100, histfunc='avg')
mean = error_df['abs_error'].mean()
std = error_df['abs_error'].std()
fig.update_layout(height=900, width=900, title_text=str(
    'Mean= ' + str(mean) + ', std=' + str(std)))
fig.show()
pio.write_image(fig, str(
    image_path + 'distribution of average prediction error on the displacement.png'), format='png')


In [ ]:
import plotly.express as px
error_df = pd.DataFrame({'errors': errors})
print(error_df.head(5))
fig = px.histogram(error_df, x='errors', nbins=50)
mean = error_df['errors'].mean()
std = error_df['errors'].std()
fig.update_layout(height=900, width=900, title_text=str(
    'Mean= ' + str(mean) + ', std=' + str(std)))
fig.show()
pio.write_image(fig, str(
    image_path + 'Distribution of the prediction error.png'), format='png')


In [ ]:
# from skl2onnx import convert_sklearn
# from skl2onnx.common.data_types import FloatTensorType
# from sklearn.pipeline import Pipeline
# from joblib import load
# pipeline= load(str(preprocessor_dir+'preprocessor.p'))
# print(pipeline)
# initial_type = [('float_input', FloatTensorType([None, 4]))]
# onx = convert_sklearn(pipeline, initial_types=initial_type)
# with open(str(preprocessor_dir + "prepeocessor.onnx"), "wb") as f:
#     f.write(onx.SerializeToString())


In [ ]:
# from joblib import dump, load
# preprocessor= load(str(preprocessor_dir+"preprocessor.p"))
# dump(preprocessor,str(preprocessor_dir+"preprocessor.p"))
